### Basic Imports

In [119]:
# Import everything and setup things
import sys
import os
file_folder = os.getcwd()
if "testing_folder" in file_folder:
    file_folder = os.path.dirname(file_folder)
    sys.path.append(file_folder)
    os.chdir(file_folder)

import datetime
import math
import time
import json
from youtube_transcript_api import YouTubeTranscriptApi
import tiktoken
import faiss
import numpy as np
import asyncio
import ipywidgets
from IPython.display import display, HTML
import imp


keys=json.load(open("working_folder/keys.json"))

# Setup anthropic
import anthropic
anthropic_client = anthropic.Anthropic(api_key=keys["anthropic"])

# Setup openai
from openai import OpenAI
os.environ["OPENAI_API_KEY"]=keys["openai"]
openai_client = OpenAI(api_key=keys["openai"])

enc=tiktoken.get_encoding("cl100k_base")
input_cost=0
output_cost=0

### Import custom library or Reload for development

In [180]:
# import and reload custom functions 
import AiFunctions as af
imp.reload(af)

<module 'AiFunctions' from 'd:\\PROGRAMS\\DestinyFolder\\DestinyAIBot\\AiFunctions.py'>

## Designate Video ID for all subsequent processes https://vyneer.me/vods/

In [175]:
video_url="https://youtu.be/savIUlLu0MY"
# Set video id
#video_id="FC3giY7YDAQ"
#video_id="hkOI46Ai4g8"
video_id="YZuSc-EnI3o"

try:
    if video_url!="":
        video_id=video_url.split("/")[-1]
except:
    pass


### Load or Get transcript

In [181]:
# Get transcript and nearest times
transcript, nearest_times=af.load_or_get_transcript_info(video_id)

Getting transcript from YouTube


### Load or Make text chunks and vector_db for transcript search

In [183]:
vector_db, text_chunks_dict = await af.load_or_get_vectordb_and_chunks(openai_client, transcript, nearest_times, video_id)

Generating text chunks and vector db
Number of chunks:  417
Finished generating embeddings


### Search Transcript

In [185]:
#load from text_chunks_dict
user_query=input("Enter your query: ")
print("Searching for:", user_query)

D, I=af.search_vector_db(openai_client, vector_db, user_query,5)

thtml=af.convert_to_html(nearest_times, video_id, text_chunks_dict[str(I[0][0])]["text"],text_chunks_dict[str(I[0][0])]["start"],I[0][0])
display(HTML(thtml))

Searching for: Elon Musk Billionaire


### Create summarized segments

In [186]:
model_responses, run_input_cost, run_output_cost=await af.load_or_make_summarized_segments(anthropic_client, transcript, nearest_times, video_id, segments=1000)
input_cost+=run_input_cost
output_cost+=run_output_cost

Approximate cost:  0.5603069999999999   Number of segments:  42
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 Error: 25 Error: 26 Error: 28 37 Error: 29 Error: 32 Error: 30 Error: 35 Error: 36 Error: 37 38 39 40 41 Retrying: 25 25 Retrying: 26 26 Retrying: 28 28 Retrying: 29 29 Retrying: 32 32 Retrying: 30 30 Error: 40 Retrying: 35 35 Retrying: 36 36 Error: 28 Error: 25 Error: 26 Retrying: 37 37 Error: 29 Error: 35 Error: 32 Error: 37 Retrying: 40 40 Retrying: 28 28 Retrying: 25 25 Retrying: 26 26 Retrying: 29 29 Retrying: 35 35 Retrying: 32 32 Error: 26 Error: 40 Error: 28 Error: 29 Retrying: 37 37 Error: 32 Error: 35 Error: 37 Retrying: 26 26 Retrying: 40 40 Retrying: 28 28 Retrying: 29 29 Retrying: 32 32 Retrying: 35 35 Retrying: 37 37 Total Cost: 0.505617
Model responses saved to file


### Make meta summary

In [188]:
meta_summary=af.load_or_make_meta_summary(anthropic_client, model_responses, video_id)

Generating meta summary
Expected cost:  0.04236
Here are two summaries for the collection of video transcript summaries you provided:

Main Topics:

1. Analysis of the Israeli-Palestinian conflict and whether Israel's actions constitute genocide against Palestinians according to the legal definition. There is in-depth discussion around proving "genocidal intent" with examples like forced mass relocations being required, versus things like civilian casualties alone. The streamer seems skeptical of the genocide claim for now based on available evidence, while remaining open to convincing proof.

2. Examination of definitions and precedents around legally determining genocide in various contexts like Russia's invasion of Ukraine, the Armenian Genocide, the Holocaust, and China's treatment of Uyghurs. The need for a very high bar of proof beyond typical wartime actions is emphasized.  

3. Concerns about the Chinese-owned app TikTok being potentially weaponized for foreign influence campai